# Get Start with BrainFlow

In [1]:
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
import time

First, initialize the board.

In [2]:
# params is the parameter to configuring the board.
# What you can do with it depends on the board you are using.
# https://brainflow.readthedocs.io/en/stable/SupportedBoards.html
# If you don't care, just leave it like that. It works.
params = BrainFlowInputParams()

# board is the object that will connect to the board.
# Since our project use UNICORN, we set the board like so.
board = BoardShim(BoardIds.UNICORN_BOARD, params)

In [3]:
# Here since I do not have UNICORN connected to my Laptop, when I prepare the session, it was not happy. 
board.prepare_session()

BrainFlowError: BOARD_NOT_READY_ERROR:7 unable to prepare streaming session

In [4]:
# For the development (or writing this document), I will use a dummy board or synthetic board.
board = BoardShim(BoardIds.SYNTHETIC_BOARD, params)

In [5]:
# This will arm the record 
board.prepare_session()
# Now we reading data from the board
board.start_stream()
# read data for 10 seconds
time.sleep(10)
# data = board.get_current_board_data (256) # get latest 256 packages or less, doesn't remove them from internal buffer
data = board.get_board_data()  # get all data and remove it from internal buffer
# stop the reading
board.stop_stream()
# unarm the record
board.release_session()

In [7]:
# This is the shape of the data
data.shape

(32, 2503)

In [8]:
# To understand the data, we use 
board_descr = BoardShim.get_board_descr(BoardIds.SYNTHETIC_BOARD)
board_descr

{'accel_channels': [17, 18, 19],
 'battery_channel': 29,
 'ecg_channels': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
 'eda_channels': [23],
 'eeg_channels': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
 'eeg_names': 'Fz,C3,Cz,C4,Pz,PO7,Oz,PO8,F5,F7,F3,F1,F2,F4,F6,F8',
 'emg_channels': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
 'eog_channels': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
 'gyro_channels': [20, 21, 22],
 'marker_channel': 31,
 'name': 'Synthetic',
 'num_rows': 32,
 'package_num_channel': 0,
 'ppg_channels': [24, 25],
 'resistance_channels': [27, 28],
 'sampling_rate': 250,
 'temperature_channels': [26],
 'timestamp_channel': 30}

In [9]:
# As you can see, the board description is depends on each board
# You can find the description of UNICORN like so.
board_descr = BoardShim.get_board_descr(BoardIds.UNICORN_BOARD)
board_descr

{'accel_channels': [8, 9, 10],
 'battery_channel': 14,
 'eeg_channels': [0, 1, 2, 3, 4, 5, 6, 7],
 'eeg_names': 'Fz,C3,Cz,C4,Pz,PO7,Oz,PO8',
 'gyro_channels': [11, 12, 13],
 'marker_channel': 18,
 'name': 'Unicorn',
 'num_rows': 19,
 'other_channels': [16],
 'package_num_channel': 15,
 'sampling_rate': 250,
 'timestamp_channel': 17}

# Add marker

Marker in BCI means when the epoch is start.
You do not need to follow the scheme if you know what you are doing.
But here is how to work with marker in normal way.

In [10]:
params = BrainFlowInputParams()
board = BoardShim(BoardIds.SYNTHETIC_BOARD, params)

board.prepare_session()
board.start_stream()

for i in range(10):
    time.sleep(1)
    board.insert_marker(i + 1)

data = board.get_board_data()
board.stop_stream()
board.release_session()

In [14]:
board_descr = BoardShim.get_board_descr(BoardIds.SYNTHETIC_BOARD)
marker_channel = board_descr['marker_channel']
data[marker_channel, data[marker_channel, :] != 0]

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

# Extracting Data

As you can see, the `data` object contains all the information provided by the board.
You may or may not like it.
Luckily, `BrainFlow` provides a method to extract data you want.

You can either do it manually with `board_descr` or use the `BoardShim.get_...` interface.

To find what are the interface/method, check this document [link](https://brainflow.readthedocs.io/en/stable/UserAPI.html#python-api-reference)

In [26]:
print("Device Name:", BoardShim.get_device_name(BoardIds.SYNTHETIC_BOARD.value))
print("EEG Channel:", BoardShim.get_eeg_channels(BoardIds.SYNTHETIC_BOARD.value))
print("Gyro Channel:", BoardShim.get_gyro_channels(BoardIds.SYNTHETIC_BOARD.value))
print("Marker Channel:", BoardShim.get_marker_channel(BoardIds.SYNTHETIC_BOARD.value))
print("Batter Status:", BoardShim.get_battery_channel(BoardIds.SYNTHETIC_BOARD.value))

Device Name: Synthetic
EEG Channel: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
Gyro Channel: [20, 21, 22]
Marker Channel: 31
Batter Status: 29


# Convert t

In [18]:
BoardShim.get

TypeError: get_eeg_channels() missing 1 required positional argument: 'board_id'

In [16]:
eeg_channels = BoardShim.get_eeg_channels(BoardIds.SYNTHETIC_BOARD.value)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]